In [60]:
#!pip install coolprop

In [61]:
import CoolProp
from CoolProp.CoolProp import PhaseSI, PropsSI, get_global_param_string
from CoolProp.HumidAirProp import HAPropsSI
print('CoolProp version: ', CoolProp.__version__)
print('CoolProp gitrevision: ', CoolProp.__gitrevision__)
print('CoolProp fluids: ', CoolProp.__fluids__)

CoolProp version:  6.4.1
CoolProp gitrevision:  1c12adfc9f981441820dab09c3d1921ac1f992c6
CoolProp fluids:  ['1-Butene', 'Acetone', 'Air', 'Ammonia', 'Argon', 'Benzene', 'CarbonDioxide', 'CarbonMonoxide', 'CarbonylSulfide', 'cis-2-Butene', 'CycloHexane', 'Cyclopentane', 'CycloPropane', 'D4', 'D5', 'D6', 'Deuterium', 'Dichloroethane', 'DiethylEther', 'DimethylCarbonate', 'DimethylEther', 'Ethane', 'Ethanol', 'EthylBenzene', 'Ethylene', 'EthyleneOxide', 'Fluorine', 'HeavyWater', 'Helium', 'HFE143m', 'Hydrogen', 'HydrogenChloride', 'HydrogenSulfide', 'IsoButane', 'IsoButene', 'Isohexane', 'Isopentane', 'Krypton', 'm-Xylene', 'MD2M', 'MD3M', 'MD4M', 'MDM', 'Methane', 'Methanol', 'MethylLinoleate', 'MethylLinolenate', 'MethylOleate', 'MethylPalmitate', 'MethylStearate', 'MM', 'n-Butane', 'n-Decane', 'n-Dodecane', 'n-Heptane', 'n-Hexane', 'n-Nonane', 'n-Octane', 'n-Pentane', 'n-Propane', 'n-Undecane', 'Neon', 'Neopentane', 'Nitrogen', 'NitrousOxide', 'Novec649', 'o-Xylene', 'OrthoDeuterium', 

In [62]:
def kelvin(T):
    return T + 273.15
def degree(T):
    return T - 273.15

class AHU:
    
    def __init__(self, P=101325.):
        self.P = P
    
    def mix(self, inputs):
        
        points_in  = inputs 
        qm_tot, qm_T, qm_w     = 0., 0., 0.
            
        for pt in points_in:
            
            pt['rho'] = PropsSI(  "D","T",kelvin(pt['T']),"P",self.P,"Air")            #kg/m3
            pt['w']   = HAPropsSI("W","T",kelvin(pt['T']),"P",self.P,"R",pt['HR']/100) #kg_w/kg_da
            pt['h']   = HAPropsSI("H","T",kelvin(pt['T']),"P",self.P,"R",pt['HR']/100) #J/kg_da
            pt['qm']  = pt['qV']*pt['rho']/3600                                        #ks/s
            qm_tot   += pt['qm']
            qm_T     += pt['qm']*pt['T']
            qm_w     += pt['qm']*pt['w']
            
        point_out = {}
        
        point_out['qm']  = qm_tot
        point_out['T']   = qm_T/qm_tot
        point_out['w']   = qm_w/qm_tot
        point_out['HR']  = HAPropsSI("R","T",kelvin(point_out['T']),"P",self.P,"W",point_out['w'])
        point_out['h']   = HAPropsSI("H","T",kelvin(point_out['T']),"P",self.P,"W",point_out['w'])
        point_out['rho'] = PropsSI(  "D","T",kelvin(point_out['T']),"P",self.P,"Air")
        point_out['qV']  = point_out['qm']/point_out['rho']*3600
        
        return points_in, point_out

In [63]:
#Example of mixing three air streams
T1       = 4.            # °C
qV1, qm1 = 1500. , None  # m3/h, ks/s
HR1, w1  = 80.   , None  # %   , kg_w/kg_da
h1       = None          # J/kg_da

T2       = 15.
qV2, qm2 = 2500. , None
HR2, w2  = 45.   , None
h2       = None

T3       = 30.
qV3, qm3 = 1100. , None
HR3, w3  = 22.   , None
h3       = None

pt1 = {'T':T1,'qV':qV1, 'qm':qm1, 'HR':HR1, 'w':w1, 'h':h1}
pt2 = {'T':T2,'qV':qV2, 'qm':qm2, 'HR':HR2, 'w':w2, 'h':h2}
pt3 = {'T':T3,'qV':qV3, 'qm':qm3, 'HR':HR3, 'w':w3, 'h':h3}

inputs = [pt1, pt2, pt3]

cta = AHU()
in_, out_ = cta.mix(inputs)

for i in range(len(inputs)):
    print('Input',i,': ',in_[i])
print('\nOutput : ',out_)

Input 0 :  {'T': 4.0, 'qV': 1500.0, 'qm': 0.53097461074087, 'HR': 80.0, 'w': 0.0040372312939574205, 'h': 14145.73565170174, 'rho': 1.2743390657780882}
Input 1 :  {'T': 15.0, 'qV': 2500.0, 'qm': 0.8510687648427115, 'HR': 45.0, 'w': 0.0047671137934510366, 'h': 27138.33348211292, 'rho': 1.2255390213735047}
Input 2 :  {'T': 30.0, 'qV': 1100.0, 'qm': 0.35589083206076294, 'HR': 22.0, 'w': 0.005813934291495015, 'h': 45041.236502471926, 'rho': 1.1647336321988606}

Output :  {'qm': 1.7379342076443445, 'T': 14.710945192845339, 'w': 0.004758485585362316, 'HR': 0.45764345819583374, 'h': 26823.15910655044, 'rho': 1.226773363008575, 'qV': 5100.015484666101}


# TO DO: THE FUNCTIONS BELOW vvvvvvvvvvvvvvvvvvvv

# Chauffage

In [5]:
#supposer sans pertes de chaleurs
pertes = 0.
calculPuis = False
calculPt_2 = True

if calculPuis:
    T_2 = 20.

if calculPt_2:
    PBC = 8015.

#m3/h
qV_1 = 1500.

#°C
T_1 = 5.

#Pa
P_1 = 101325.

#%
HR_1 = 40.

#kg/m3
rho_1 = PropsSI("D","T",kelvin(T_1),"P",P_1,"Air")

#kg_w/kg_da
w_1 = HAPropsSI("W","T",kelvin(T_1),"P",P_1,"R",HR_1/100)

#J/kg_da
h_1 = HAPropsSI("H","T",kelvin(T_1),"P",P_1,"R",HR_1/100)

#ks/s
qm_1 = qV_1*rho_1/3600

#RESOLUTION
qm_2 = qm_1
w_2  = w_1 
P_2  = P_1

if calculPuis:    
    h_2 = HAPropsSI("H","T",kelvin(T_2),"P",P_2,"W",w_2)
    PBC = qm_2*(h_2-h_1)+pertes
    
if calculPt_2:
    h_2 = (PBC - pertes)/qm_2 + h_1
    T_2 = degree(HAPropsSI("T","H",h_2,"P",P_2,"W",w_2))
    
HR_2  = HAPropsSI("R","T",kelvin(T_2),"P",P_2,"W",w_2)*100.
rho_2 = PropsSI("D","T",kelvin(T_2),"P",P_2,"Air")
qV_2  = qm_2/rho_2*3600

print('Point 1:')
print('qV 1: ',qV_1,' m3/h')
print('qm 1: ',qm_1,' kg/s')
print('T 1: ',T_1,' °C')
print('P 1: ',P_1,' Pa')
print('HR 1: ',HR_1,' %')
print('w 1: ',w_1,' kg_w/kg_da')
print('h 1: ',h_1,' J/kg_da')
print(' ')
print('Point 2:')
print('qV 2: ',qV_2,' m3/h')
print('qm 2: ',qm_2,' kg/s')
print('T 2: ',T_2,' °C')
print('P 2: ',P_2,' Pa')
print('HR 2: ',HR_2,' %')
print('w 2: ',w_2,' kg_w/kg_da')
print('h 2: ',h_2,' J/kg_da')
print(' ')
print('Batterie chaude:')
print('P: ',PBC,' W')
print('Pertes: ',pertes,' W')

Point 1:
qV 1:  1500.0  m3/h
qm 1:  0.5290592311149505  kg/s
T 1:  5.0  °C
P 1:  101325.0  Pa
HR 1:  40.0  %
w 1:  0.0021585653358280697  kg_w/kg_da
h 1:  10445.418318573726  J/kg_da
 
Point 2:
qV 2:  1581.14951696535  m3/h
qm 2:  0.5290592311149505  kg/s
T 2:  20.000036210259736  °C
P 2:  101325.0  Pa
HR 2:  14.91918945594993  %
w 2:  0.0021585653358280697  kg_w/kg_da
h 2:  25594.950787951533  J/kg_da
 
Batterie chaude:
P:  8015.0  W
Pertes:  0.0  W


# Refroidissement sans deshumidification

In [6]:
#supposer sans pertes de chaleurs
apports = 0.
calculPuis = False
calculPt_2 = True

if calculPuis:
    T_2 = 5.

if calculPt_2:
    PBF = 8015.

#m3/h
qV_1 = 1581.

#°C
T_1 = 20.

#Pa
P_1 = 101325.

#%
HR_1 = 15.

#kg/m3
rho_1 = PropsSI("D","T",kelvin(T_1),"P",P_1,"Air")

#kg_w/kg_da
w_1 = HAPropsSI("W","T",kelvin(T_1),"P",P_1,"R",HR_1/100)

#J/kg_da
h_1 = HAPropsSI("H","T",kelvin(T_1),"P",P_1,"R",HR_1/100)

#ks/s
qm_1 = qV_1*rho_1/3600

#RESOLUTION
qm_2 = qm_1
w_2  = w_1 
P_2  = P_1

if calculPuis:    
    h_2 = HAPropsSI("H","T",kelvin(T_2),"P",P_2,"W",w_2)
    PBF = qm_2*(h_1-h_2)+apports
    
if calculPt_2:
    h_2 = h_1-(PBF - apports)/qm_2
    T_2 = degree(HAPropsSI("T","H",h_2,"P",P_2,"W",w_2))
    
HR_2  = HAPropsSI("R","T",kelvin(T_2),"P",P_2,"W",w_2)*100.
rho_2 = PropsSI("D","T",kelvin(T_2),"P",P_2,"Air")
qV_2  = qm_2/rho_2*3600

print('Point 1:')
print('qV 1: ',qV_1,' m3/h')
print('qm 1: ',qm_1,' kg/s')
print('T 1: ',T_1,' °C')
print('P 1: ',P_1,' Pa')
print('HR 1: ',HR_1,' %')
print('w 1: ',w_1,' kg_w/kg_da')
print('h 1: ',h_1,' J/kg_da')
print(' ')
print('Point 2:')
print('qV 2: ',qV_2,' m3/h')
print('qm 2: ',qm_2,' kg/s')
print('T 2: ',T_2,' °C')
print('P 2: ',P_2,' Pa')
print('HR 2: ',HR_2,' %')
print('w 2: ',w_2,' kg_w/kg_da')
print('h 2: ',h_2,' J/kg_da')
print(' ')
print('Batterie froide:')
print('P: ',PBF,' W')
print('Apports: ',pertes,' W')

Point 1:
qV 1:  1581.0  m3/h
qm 1:  0.5290092676449086  kg/s
T 1:  20.0  °C
P 1:  101325.0  Pa
HR 1:  15.0  %
w 1:  0.002170293227330943  kg_w/kg_da
h 1:  25624.67267929687  J/kg_da
 
Point 2:
qV 2:  1499.85225034107  m3/h
qm 2:  0.5290092676449086  kg/s
T 2:  4.9988739815901795  °C
P 2:  101325.0  Pa
HR 2:  40.219730714445724  %
w 2:  0.002170293227330943  kg_w/kg_da
h 2:  10473.709378253197  J/kg_da
 
Batterie froide:
P:  8015.0  W
Apports:  0.0  W


# Refroidissement et deshumidification de l'air (BTF hum.)

In [7]:
#calcul T_ms
ff   = False
eau  = True

if ff: #battarie a detente directe à ff
    T_evap = 5.
    T_ms   = T_evap+4 #+4 a 6deg
if eau:
    T_e = 0. #entree BF
    T_s = 5. #sortie BF
    T_ms = (T_e+T_s)/2

In [8]:
#supposer sans pertes de chaleurs
apports = 0.
calculPuis = False #False True
calculPt_2 = False  #False True
calculWithTms  = True  #False True

if calculPuis:
    T_2  = 5.
    HR_2 = 15.

if calculPt_2:
    PBF  = 9823.
    qm_0 = 0.000728191857777296
    
if calculWithTms:
    T_ms = -10.
    T_2  = 15.

#m3/h
qV_1 = 1581.

#°C
T_1 = 20.

#Pa
P_1 = 101325.

#%
HR_1 = 15.

#kg/m3
rho_1 = PropsSI("D","T",kelvin(T_1),"P",P_1,"Air")

#kg_w/kg_da
w_1 = HAPropsSI("W","T",kelvin(T_1),"P",P_1,"R",HR_1/100)

#J/kg_da
h_1 = HAPropsSI("H","T",kelvin(T_1),"P",P_1,"R",HR_1/100)

#ks/s
qm_1 = qV_1*rho_1/3600

#RESOLUTION
qm_2 = qm_1 #approx juste
P_2  = P_1

if calculPuis: #ajouter ms    
    w_2 = HAPropsSI("W","T",kelvin(T_2),"P",P_2,"R",HR_2/100)
    h_2 = HAPropsSI("H","T",kelvin(T_2),"P",P_2,"W",w_2)
    PBF = qm_2*(h_1-h_2)+apports
    
    #debit condensats
    qm_0 = qm_1*(w_1-w_2)
    
if calculWithTms: #verifier ce calcul
    h_ms = HAPropsSI("H","T",kelvin(T_ms),"P",P_1,"R",1.)
    w_ms = HAPropsSI("W","T",kelvin(T_ms),"P",P_1,"R",1.)
    
    h_2  = (h_1-h_ms)/(T_1-T_ms)*T_2    
    w_2  = (w_1-w_ms)/(T_1-T_ms)*T_2
    HR_2 = w_2/HAPropsSI("W","T",kelvin(T_2),"P",P_2,"R",1.)*100
    PBF  = qm_2*(h_1-h_2)+apports
    
    #debit condensats
    qm_0 = qm_1*(w_1-w_2)    
    
if calculPt_2: #ajouter ms
    h_2 = h_1-(PBF - apports)/qm_2
    w_2 = w_1-qm_0/qm_1
    T_2 = degree(HAPropsSI("T","H",h_2,"P",P_2,"W",w_2))
    HR_2 = HAPropsSI("R","T",kelvin(T_2),"P",P_2,"W",w_2)*100
    
rho_2 = PropsSI("D","T",kelvin(T_2),"P",P_2,"Air")
qV_2  = qm_2/rho_2*3600

h_3 = HAPropsSI("H","T",kelvin(T_1),"P",P_1,"W",w_2)

Qlat  = (h_1-h_3)
Qsens = (h_3-h_2)

#### AJOUTER EFFICACITE ET TEMP INFINI

print('Point 1:')
print('qV 1: ',qV_1,' m3/h')
print('qm 1: ',qm_1,' kg/s')
print('T 1: ',T_1,' °C')
print('P 1: ',P_1,' Pa')
print('HR 1: ',HR_1,' %')
print('w 1: ',w_1,' kg_w/kg_da')
print('h 1: ',h_1,' J/kg_da')
print(' ')
print('Point 2:')
print('qV 2: ',qV_2,' m3/h')
print('qm 2: ',qm_2,' kg/s')
print('T 2: ',T_2,' °C')
print('P 2: ',P_2,' Pa')
print('HR 2: ',HR_2,' %')
print('w 2: ',w_2,' kg_w/kg_da')
print('h 2: ',h_2,' J/kg_da')
print(' ')
print('Batterie froide:')
print('P: ',PBF,' W')
print('Q lat: ',Qlat,' J/kg_da')
print('Q sens: ',Qsens,' J/kg_da')
print('Debit condensat: ',qm_0,' kg_e/s')
print('Apports: ',pertes,' W')

Point 1:
qV 1:  1581.0  m3/h
qm 1:  0.5290092676449086  kg/s
T 1:  20.0  °C
P 1:  101325.0  Pa
HR 1:  15.0  %
w 1:  0.002170293227330943  kg_w/kg_da
h 1:  25624.67267929687  J/kg_da
 
Point 2:
qV 2:  1553.9557127991777  m3/h
qm 2:  0.5290092676449086  kg/s
T 2:  15.0  °C
P 2:  101325.0  Pa
HR 2:  2.637481792591117  %
w 2:  0.00028204615737633266  kg_w/kg_da
h 2:  15847.486888418432  J/kg_da
 
Batterie froide:
P:  5172.221894860809  W
Q lat:  4791.530632302696  J/kg_da
Q sens:  4985.655158575741  J/kg_da
Debit condensat:  0.000998900199609333  kg_e/s
Apports:  0.0  W


# DESHUMIDIFICATION absorbant liq

In [9]:
#ce qu'on veut
HR_2 = 20.
refr = 2. #voir amélioration

#m3/h
qV_1 = 1581.

#°C
T_1 = 30.
T_2 = T_1-refr

#Pa
P_1 = 101325.

#%
HR_1 = 60.

#kg/m3
rho_1 = PropsSI("D","T",kelvin(T_1),"P",P_1,"Air")

#kg_w/kg_da
w_1 = HAPropsSI("W","T",kelvin(T_1),"P",P_1,"R",HR_1/100)

#J/kg_da
h_1 = HAPropsSI("H","T",kelvin(T_1),"P",P_1,"R",HR_1/100)

#ks/s
qm_1 = qV_1*rho_1/3600

#RESOLUTION
qm_2 = qm_1 #approx juste
P_2  = P_1

#debit condensats
w_2 = HAPropsSI("W","T",kelvin(T_2),"P",P_2,"R",HR_2/100)
qm_0 = qm_1*(w_1-w_2)

h_2 = HAPropsSI("H","T",kelvin(T_2),"P",P_2,"W",w_2)
    
rho_2 = PropsSI("D","T",kelvin(T_2),"P",P_2,"Air")
qV_2  = qm_2/rho_2*3600

#### AJOUTER EFFICACITE ET TEMP INFINI

print('Point 1:')
print('qV 1: ',qV_1,' m3/h')
print('qm 1: ',qm_1,' kg/s')
print('T 1: ',T_1,' °C')
print('P 1: ',P_1,' Pa')
print('HR 1: ',HR_1,' %')
print('w 1: ',w_1,' kg_w/kg_da')
print('h 1: ',h_1,' J/kg_da')
print(' ')
print('Point 2:')
print('qV 2: ',qV_2,' m3/h')
print('qm 2: ',qm_2,' kg/s')
print('T 2: ',T_2,' °C')
print('P 2: ',P_2,' Pa')
print('HR 2: ',HR_2,' %')
print('w 2: ',w_2,' kg_w/kg_da')
print('h 2: ',h_2,' J/kg_da')
print(' ')
print('Condensat:')
print('Debit condensat: ',qm_0,' kg_e/s')

Point 1:
qV 1:  1581.0  m3/h
qm 1:  0.5115121868073329  kg/s
T 1:  30.0  °C
P 1:  101325.0  Pa
HR 1:  60.0  %
w 1:  0.01611640841360815  kg_w/kg_da
h 1:  71365.22098706919  J/kg_da
 
Point 2:
qV 2:  1570.5424081947233  m3/h
qm 2:  0.5115121868073329  kg/s
T 2:  28.0  °C
P 2:  101325.0  Pa
HR 2:  20.0  %
w 2:  0.004699401635617976  kg_w/kg_da
h 2:  40162.20083052669  J/kg_da
 
Condensat:
Debit condensat:  0.005839938103803895  kg_e/s


# DESHUMIDIFICATION absorbant solide

In [10]:
#ce qu'on veut
HR_2 = 20.

#m3/h
qV_1 = 1581.

#°C
T_1 = 20.

#Pa
P_1 = 101325.

#%
HR_1 = 90.

#kg/m3
rho_1 = PropsSI("D","T",kelvin(T_1),"P",P_1,"Air")

#kg_w/kg_da
w_1 = HAPropsSI("W","T",kelvin(T_1),"P",P_1,"R",HR_1/100)

#J/kg_da
h_1 = HAPropsSI("H","T",kelvin(T_1),"P",P_1,"R",HR_1/100)
h_2 = h_1 # approx ok

#ks/s
qm_1 = qV_1*rho_1/3600

#RESOLUTION
qm_2 = qm_1 #approx juste
P_2  = P_1

#debit condensats
w_2 = HAPropsSI("W","H",h_2,"P",P_2,"R",HR_2/100)
qm_0 = qm_1*(w_1-w_2)

T_2 = degree(HAPropsSI("T","H",h_2,"P",P_2,"W",w_2))
    
rho_2 = PropsSI("D","T",kelvin(T_2),"P",P_2,"Air")
qV_2  = qm_2/rho_2*3600

#### AJOUTER EFFICACITE ET TEMP INFINI

print('Point 1:')
print('qV 1: ',qV_1,' m3/h')
print('qm 1: ',qm_1,' kg/s')
print('T 1: ',T_1,' °C')
print('P 1: ',P_1,' Pa')
print('HR 1: ',HR_1,' %')
print('w 1: ',w_1,' kg_w/kg_da')
print('h 1: ',h_1,' J/kg_da')
print(' ')
print('Point 2:')
print('qV 2: ',qV_2,' m3/h')
print('qm 2: ',qm_2,' kg/s')
print('T 2: ',T_2,' °C')
print('P 2: ',P_2,' Pa')
print('HR 2: ',HR_2,' %')
print('w 2: ',w_2,' kg_w/kg_da')
print('h 2: ',h_2,' J/kg_da')
print(' ')
print('Condensat:')
print('Debit condensat: ',qm_0,' kg_e/s')

Point 1:
qV 1:  1581.0  m3/h
qm 1:  0.5290092676449086  kg/s
T 1:  20.0  °C
P 1:  101325.0  Pa
HR 1:  90.0  %
w 1:  0.013252992674350803  kg_w/kg_da
h 1:  53736.69863043071  J/kg_da
 
Point 2:
qV 2:  1663.4836838671486  m3/h
qm 2:  0.5290092676449086  kg/s
T 2:  35.25272140222427  °C
P 2:  101325.0  Pa
HR 2:  20.0  %
w 2:  0.007120003269781268  kg_w/kg_da
h 2:  53736.69863043071  J/kg_da
 
Condensat:
Debit condensat:  0.0032444082333853135  kg_e/s
